# DocTAPE

DocTAPE (Documentation Testing and Automated Placement of Expressions) is a collection of utility functions (and wrappers for [Glue](https://myst-nb.readthedocs.io/en/latest/render/glue.html)) that are useful
for automating the process of building and testing documentation to ensure that
documentation doesn't get stale.

Our standard practice it to include a comment (`# Testing Cell`) at the begining of code cells as well as make use of the `remove-cell` tag.

>   "metadata": { "tags": [ "remove-cell" ] },


In [ ]:
# Testing Cell


In [39]:
# Testing Cell

from aviary.docs.tests import utils
import inspect

imported_functions = {k:v for k,v in inspect.getmembers(utils, inspect.isfunction) if v.__module__ == utils.__name__}
imported_classes = {k:v for k,v in inspect.getmembers(utils, inspect.isclass) if v.__module__ == utils.__name__}

custom_classes = {
    "expected_error": "is an execption that can be used in try/except blocks to allow desired errors to pass while still raising unexpected errors.",
}
testing_functions = {
    "check_value": "is a simple function for comparing two values",
    "check_contains": "confirms that all the elements of one iterable are contained in the other",
    "check_args": "gets the signature of a function and compares it to the arguments you are expecting",
    "run_command_no_file_error": "executes a CLI command but won't fail if a FileNotFoundError is raised",
}
utility_functions = {
    "gramatical_list": "combines the elements of a list into a string with proper punctuation",
    "get_attribute_name": "gets the name of an object's attribute based on it's value",
    "get_all_keys": "recursively get all of the keys from a dict of dicts",
    "get_value": "recursively get a value from a dict of dicts",
}
glue_functions = {
    "glue_variable": "Glue a variable for later use in markdown cells of notebooks (can auto format for code)",
    "glue_keys": "recursively glue all of the keys from a dict of dicts",
}

utils.check_value(imported_classes.keys(),custom_classes.keys())
utils.check_value(imported_functions.keys(), {
    **testing_functions, **glue_functions, **utility_functions}.keys())

class_list = ''
for key,val in custom_classes.items():
    utils.glue_variable(key, md_code=True)
    class_list += f'- `{key}` {val}\n'

# testing_list = ''
# for key,val in testing_functions.items():
#     testing_list += f'- `{key}` {val}\n'

utility_list = '```{eval-rst}\n'
for key in utility_functions:
    utils.glue_variable(key, md_code=True)
    utility_list += ' '*4+f'.. autofunction:: aviary.docs.tests.utils.{key}\n{" "*8}:noindex:\n\n'
utility_list += '```'

# testing_list = '```{eval-rst}\n'
# for key in testing_functions:
#     utils.glue_variable(key, md_code=True)
#     testing_list += ' '*4+f'.. autofunction:: aviary.docs.tests.utils.{key}\n{" "*8}:noindex:\n\n'
# testing_list += '```'

testing_list = '<details>\n\n<summary>Function Docs</summary>\n\n'
testing_list += '```{eval-rst}\n'
for key in testing_functions:
    utils.glue_variable(key, md_code=True)
    testing_list += ' '*4+f'.. autofunction:: aviary.docs.tests.utils.{key}\n{" "*8}:noindex:\n\n'
testing_list += '```'
testing_list += '\n\n</details>'

# glue_list = ''
# for key,val in glue_functions.items():
#     glue_list += f'- `{key}` {key}\n'

# glue_list = ''
# for key in glue_functions:
#     # doc_str = inspect.getdoc(imported_functions[key])
#     doc_str = imported_functions[key].__doc__.split('\n')[1]
#     # doc_str = '\n'.join([s+'  ' for s in imported_functions[key].__doc__.split('\n')])
#     print(doc_str)
#     glue_list += f'- `{key}`: {doc_str}\n'

glue_list = '```{eval-rst}\n'
for key in glue_functions:
    utils.glue_variable(key, md_code=True)
    glue_list += ' '*4+f'.. autofunction:: aviary.docs.tests.utils.{key}\n{" "*8}:noindex:\n\n'
glue_list += '```'

utils.glue_variable('class_list', utils.Markdown(class_list))
utils.glue_variable('utility_list', utils.Markdown(utility_list))
utils.glue_variable('testing_list', utils.Markdown(testing_list))
utils.glue_variable('glue_list', utils.Markdown(glue_list))

```{eval-rst}
    .. autofunction:: aviary.docs.tests.utils.glue_variable
        :noindex:

    .. autofunction:: aviary.docs.tests.utils.glue_keys
        :noindex:

```

## Classes
```{glue:md} class_list
:format: myst
```

## Testing Functions

Functions that raise an error provide the option to specify an error type to use instead of the default. This allows users to change the error type that is raised which can be useful in try/except blocks, especially when combined with the {glue:md}`expected_error` class.

In [12]:
from aviary.docs.tests.utils import expected_error, check_value
try:
    check_value(int('1'), 2, error_type=expected_error)
except expected_error:
    print('we expected that to fail (1 is not equal to 2),')
print('but this will still run')

we expected that to fail (1 is not equal to 2),
but this will still run


If we just used `ValueError` in the `except` branch, we might miss errors that we actually do want to catch.

In [16]:
from aviary.docs.tests.utils import expected_error, check_value

try:
    check_value(int('1)'), 2)
except ValueError:
    print('1 is not equal to 2')
print("we mistyped '1', so we should have failed")

try:
    check_value(int('1)'), 2, error_type=expected_error)
except expected_error:
    print('1 is not equal to 2')
print("something unnexpected happened (we mistyped '1'), and we won't reach this")

1 is not equal to 2
we mistyped '1', so we should have failed


ValueError: invalid literal for int() with base 10: '1)'

```{glue:md} testing_list
:format: myst
```

## Utility Functions

Utility functions are provided that the user may find useful for generating or testing their documentation.

{glue:md}`gramatical_list` is a simple function that forms a string that can be used in a sentence using a list of items.

In [18]:
from aviary.docs.tests.utils import gramatical_list

single_element = gramatical_list([1])
two_elements = gramatical_list(['apples','bananas'])
three_elements_with_or = gramatical_list(['apples','bananas', 'strawberries'],'or')

print(f"I would like to order {single_element} smoothie.")
print(f"Do you want {three_elements_with_or} in your smoothie?")
print(f"I only want {two_elements}.")

I would like to order 1 smoothie.
Do you want apples, bananas, or strawberries in your smoothie?
I only want apples and bananas.


{glue:md}`get_attribute_name` allows users to get the name of object attributes in order to glue them into documentation. This works well for Enums  or Class Variables that have unique values.

In [30]:
from aviary.docs.tests.utils import get_attribute_name, glue_variable
from aviary.api import LegacyCode
import aviary.api as av

some_custom_alias = av.LegacyCode

gasp_name = get_attribute_name(some_custom_alias, LegacyCode.GASP)
glue_variable(gasp_name)
brief_name = get_attribute_name(av.Verbosity, 1)
glue_variable(brief_name)
verbosity = get_attribute_name(av.Settings, av.Settings.VERBOSITY)
glue_variable(verbosity)

'GASP'

'BRIEF'

'VERBOSITY'

{glue:md}`get_all_keys` and {glue:md}`get_value` are intended to be used together for getting keys from nested dictionaries and then getting values back from those nested dictionaries, respectively. They were originally added for complex dictionaries, like the phase_info.

In [33]:
from aviary.docs.tests.utils import get_all_keys, get_value

simplified_dict = {
    'phase1':{'altitude':{'val':30,'units':'kft'},'mach':.4},
    'phase2':{'altitude':{'val':10,'units':'km'},'mach':.5}
    }
unique_keys_only = get_all_keys(simplified_dict)
all_keys = get_all_keys(simplified_dict, track_layers=True)
print(unique_keys_only)
print(all_keys)

p1_alt = get_value(simplified_dict, 'phase1.altitude.val')
print(p1_alt)

['phase1', 'altitude', 'val', 'units', 'mach', 'phase2']
['phase1', 'phase1.altitude', 'phase1.altitude.val', 'phase1.altitude.units', 'phase1.mach', 'phase2', 'phase2.altitude', 'phase2.altitude.val', 'phase2.altitude.units', 'phase2.mach']
30


```{glue:md} utility_list
:format: myst
```

## Glue Functions
```{glue:md} glue_list
:format: myst
```